In [1]:
!pip install pysmiles
!pip install pyTDC

  Using cached pbr-5.6.0-py2.py3-none-any.whl (111 kB)
  Created wheel for pysmiles: filename=pysmiles-1.0.1-py2.py3-none-any.whl size=22028 sha256=62a4c5c9c850088e52b4e3a47cd5ca25b6c853d182620c5bd7c27beab74bedad
  Stored in directory: /root/.cache/pip/wheels/e4/f0/ca/dae2e932684a6e26824d29cf5b6dadea7320e6fed036942972
Successfully built pysmiles
     |████████████████████████████████| 52 kB 964 kB/s 
  Created wheel for pyTDC: filename=PyTDC-0.3.0-py3-none-any.whl size=81385 sha256=ab3667ee696e6f91d46c81beb02d1311cce8c7271b8edf4661522ca197aa484f
  Stored in directory: /root/.cache/pip/wheels/27/21/34/70d74583f2bf248f91fe1abfe46af06a17371067505d2fc5c0
Successfully built pyTDC


In [2]:
import networkx as nx
import numpy as np
import pandas as pd
from pysmiles import read_smiles

from tdc.single_pred import ADME
import copy

# Psudo Code

Color refinement with different starting colors

There are a couple of things to keep in mind. First, You want to keep the number of unique colors limited. 

If you use somethign like:

Node color (i+1) =  hash (node color (i) ~append? multiply? add? (color for color in Node neighbors) % N You are garenteed to have at most N colors. Where you run into problems when mutiplue colors are hashed together. 

Odds of a spurious hash for a single node == 1/N

Odds of a spurious hash for a single iteration == (num nodes) / N

Odds of a spurious hash for k iteratiosn = K *(num_nodes) / N

### I have not double checked this math. I should run it by someone who knows it better. You should run some simulations on this.


Requirements


1. Given a graph where each vertex is garenteed to have exactly one attribute that can be treated as a int. 





In [12]:
g = read_smiles('COc1ccccc1CNC(=O)Cn1nnc(-c2ccncc2)n1')


In [102]:
def get_next_node_color(node_color:int, neighbor_colors:list):
  """
    Generate node Color i+1 based on node color i and the neighbor colors the hash function, and mod num_buckets
  """
  num_buckets = 100
  # neighborcolrs is unordered so all lists should map into the same int, then they should be appended to the orgional color then you hash that

  pre_hash_color = str(node_color) + str(np.prod(neighbor_colors)) # You need a determanistic way to evenly put this into the buckets. str nab

  return hash(pre_hash_color) % num_buckets
  # i think this is good enough. You should test it empericaly. bootstrap a bunch of combinations



get_next_node_color(11,[2,9,5])


28

In [68]:
# initla colors 

a = dict(g.nodes.data(True))


type(a)

dict

In [93]:
node_names = list(a.keys())
node_names

node_attributes = tuple(sorted(list(a[node_names[0]].items())))
node_attributes

node_starting_colors = {}
num_buckets =100

for index, node_num in enumerate(node_names):
  node_attributes = tuple(sorted(list(a[node_names[index]].items())))
  inital_node_color = hash(node_attributes) % num_buckets
  node_starting_colors[node_num] = inital_node_color


node_starting_colors

{0: 43,
 1: 39,
 2: 15,
 3: 40,
 4: 40,
 5: 40,
 6: 40,
 7: 15,
 8: 68,
 9: 45,
 10: 18,
 11: 39,
 12: 68,
 13: 45,
 14: 70,
 15: 70,
 16: 18,
 17: 15,
 18: 40,
 19: 40,
 20: 31,
 21: 40,
 22: 40,
 23: 70}

In [95]:
a

{0: {'aromatic': False, 'charge': 0, 'element': 'C', 'hcount': 3},
 1: {'aromatic': False, 'charge': 0, 'element': 'O', 'hcount': 0},
 2: {'aromatic': True, 'charge': 0, 'element': 'C', 'hcount': 0},
 3: {'aromatic': True, 'charge': 0, 'element': 'C', 'hcount': 1},
 4: {'aromatic': True, 'charge': 0, 'element': 'C', 'hcount': 1},
 5: {'aromatic': True, 'charge': 0, 'element': 'C', 'hcount': 1},
 6: {'aromatic': True, 'charge': 0, 'element': 'C', 'hcount': 1},
 7: {'aromatic': True, 'charge': 0, 'element': 'C', 'hcount': 0},
 8: {'aromatic': False, 'charge': 0, 'element': 'C', 'hcount': 2},
 9: {'aromatic': False, 'charge': 0, 'element': 'N', 'hcount': 1},
 10: {'aromatic': False, 'charge': 0, 'element': 'C', 'hcount': 0},
 11: {'aromatic': False, 'charge': 0, 'element': 'O', 'hcount': 0},
 12: {'aromatic': False, 'charge': 0, 'element': 'C', 'hcount': 2},
 13: {'aromatic': False, 'charge': 0, 'element': 'N', 'hcount': 1},
 14: {'aromatic': False, 'charge': 0, 'element': 'N', 'hcount': 

In [123]:
def convert_node_view_to_node_color_dict(node_data: nx.classes.reportviews.NodeDataView, num_buckets: int)-> dict:
  """
      Set the inital color of each of the nodes based on hashing the node attributes and a number of buckets. 
        ### add a limit based on the node feature, pass this a list of keys to encode as attributes
      # wrap this is a factory method. 
      Returns a dictionary of inital colors to make the nodes
  """
  node_data_as_dict = dict(node_data)
  node_names = list(node_data_as_dict.keys())
  node_starting_colors = dict()

  for index, node_num in enumerate(node_names):
    node_attributes = tuple(sorted(list(a[node_names[index]].items())))
    inital_node_color = hash(node_attributes) % num_buckets
    node_starting_colors[node_num] = inital_node_color

  return node_starting_colors


starting_colors = convert_node_view_to_node_color_dict(g.nodes.data(data=True), 100)
starting_colors

{0: 43,
 1: 39,
 2: 15,
 3: 40,
 4: 40,
 5: 40,
 6: 40,
 7: 15,
 8: 68,
 9: 45,
 10: 18,
 11: 39,
 12: 68,
 13: 45,
 14: 70,
 15: 70,
 16: 18,
 17: 15,
 18: 40,
 19: 40,
 20: 31,
 21: 40,
 22: 40,
 23: 70}

In [130]:
copy_g = copy.deepcopy(g)

nx.set_node_attributes(copy_g, starting_colors, name="color")
copy_g.nodes.data(True)

NodeDataView({0: {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 3, 'color': 43}, 1: {'element': 'O', 'charge': 0, 'aromatic': False, 'hcount': 0, 'color': 39}, 2: {'element': 'C', 'charge': 0, 'aromatic': True, 'hcount': 0, 'color': 15}, 3: {'element': 'C', 'charge': 0, 'aromatic': True, 'hcount': 1, 'color': 40}, 4: {'element': 'C', 'charge': 0, 'aromatic': True, 'hcount': 1, 'color': 40}, 5: {'element': 'C', 'charge': 0, 'aromatic': True, 'hcount': 1, 'color': 40}, 6: {'element': 'C', 'charge': 0, 'aromatic': True, 'hcount': 1, 'color': 40}, 7: {'element': 'C', 'charge': 0, 'aromatic': True, 'hcount': 0, 'color': 15}, 8: {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 2, 'color': 68}, 9: {'element': 'N', 'charge': 0, 'aromatic': False, 'hcount': 1, 'color': 45}, 10: {'element': 'C', 'charge': 0, 'aromatic': False, 'hcount': 0, 'color': 18}, 11: {'element': 'O', 'charge': 0, 'aromatic': False, 'hcount': 0, 'color': 39}, 12: {'element': 'C', 'charge': 0, 'aromati

In [255]:
def convert_node_view_to_node_color_dict(node_data: nx.classes.reportviews.NodeDataView, num_buckets: int)-> dict:
  """
      Set the inital color of each of the nodes based on hashing the node attributes and a number of buckets. 
        ### add a limit based on the node feature, pass this a list of keys to encode as attributes
      # wrap this is a factory method. 
      Returns a dictionary of inital colors to make the nodes
  """

  node_data_as_dict = dict(node_data)
  node_names = list(node_data_as_dict.keys())
  node_starting_colors = dict()

  for index, node_num in enumerate(node_names):
    node_attributes = tuple(sorted(list(a[node_names[index]].items())))
    inital_node_color = hash(node_attributes) % num_buckets
    node_starting_colors[node_num] = inital_node_color

  return node_starting_colors

def create_color_graph(graph: nx.classes.graph.Graph):

  starting_colors = convert_node_view_to_node_color_dict(graph.nodes.data(True), 100)
  color_graph = copy.deepcopy(graph)
  nx.set_node_attributes(color_graph, starting_colors, name="color_0")

  # remove the other only keep 'color' attribute
  for index, node in enumerate(color_graph.nodes.data(True)):
    node_attributes = list(color_graph.nodes[index].keys())
    attributes_to_remove = [a for a in node_attributes if a !='color_0']
    [color_graph.nodes[index].pop(a) for a in attributes_to_remove]

  return color_graph

c = create_color_graph(g)
c.nodes.data(True)

NodeDataView({0: {'color_0': 43}, 1: {'color_0': 39}, 2: {'color_0': 15}, 3: {'color_0': 40}, 4: {'color_0': 40}, 5: {'color_0': 40}, 6: {'color_0': 40}, 7: {'color_0': 15}, 8: {'color_0': 68}, 9: {'color_0': 45}, 10: {'color_0': 18}, 11: {'color_0': 39}, 12: {'color_0': 68}, 13: {'color_0': 45}, 14: {'color_0': 70}, 15: {'color_0': 70}, 16: {'color_0': 18}, 17: {'color_0': 15}, 18: {'color_0': 40}, 19: {'color_0': 40}, 20: {'color_0': 31}, 21: {'color_0': 40}, 22: {'color_0': 40}, 23: {'color_0': 70}})

70

In [272]:
def convert_node_view_to_node_color_dict(node_data: nx.classes.reportviews.NodeDataView, num_buckets: int)-> dict:
  """
      Set the inital color of each of the nodes based on hashing the node attributes and a number of buckets. 
        ### add a limit based on the node feature, pass this a list of keys to encode as attributes
      # wrap this is a factory method. 
      Returns a dictionary of inital colors to make the nodes
  """

  node_data_as_dict = dict(node_data)
  node_names = list(node_data_as_dict.keys())
  node_starting_colors = dict()

  for index, node_num in enumerate(node_names):
    node_attributes = tuple(sorted(list(a[node_names[index]].items())))
    inital_node_color = hash(node_attributes) % num_buckets
    node_starting_colors[node_num] = inital_node_color

  return node_starting_colors

def create_inital_color_graph(graph: nx.classes.graph.Graph):

  starting_colors = convert_node_view_to_node_color_dict(graph.nodes.data(True), 100)
  color_graph = copy.deepcopy(graph)
  nx.set_node_attributes(color_graph, starting_colors, name="color_0")

  # remove the other only keep 'color' attribute
  for index, node in enumerate(color_graph.nodes.data(True)):
    node_attributes = list(color_graph.nodes[index].keys())
    attributes_to_remove = [a for a in node_attributes if a !='color_0']
    [color_graph.nodes[index].pop(a) for a in attributes_to_remove]

  return color_graph



def compute_next_iteration_color(node_color:int, neighbor_colors:list):
  """
    Generate node Color i+1 based on node color i and the neighbor colors the hash function, and mod num_buckets
      # neighborcolrs is unordered so all lists should map into the same int, then they should be appended to the orgional color then you hash that
  """
  num_buckets = 100 
  neighbor_colors = sorted(neighbor_colors)
  pre_hash_color = str(node_color) + str(neighbor_colors)
  new_color = hash(pre_hash_color) % num_buckets 
  return new_color


def next_iteration_of_color_graph(color_graph, iter_num:int):
  prev_color_key = f'color_{iter_num-1}'
  
  # do an iteration
  updated_colors = dict()
  for index, current_node in enumerate(color_graph.nodes.data(True)):
    neighbor_nodes = [n for n in color_graph.neighbors(index)]
    current_color = color_graph.nodes[current_node[0]]
    neighbor_colors = ([color_graph.nodes[n][prev_color_key] for n in neighbor_nodes])
    updated_colors[index] = compute_next_iteration_color(current_color,neighbor_colors)

  nx.set_node_attributes(color_graph, updated_colors, name=f'color_{iter_num}')




c = create_inital_color_graph(g)

for iter_num in range(1,10):
  next_iteration_of_color_graph(c,iter_num)

c.nodes.data(True)

NodeDataView({0: {'color_0': 43, 'color_1': 85, 'color_2': 68, 'color_3': 69, 'color_4': 94, 'color_5': 5, 'color_6': 80, 'color_7': 21, 'color_8': 16, 'color_9': 77}, 1: {'color_0': 39, 'color_1': 61, 'color_2': 74, 'color_3': 7, 'color_4': 21, 'color_5': 55, 'color_6': 24, 'color_7': 11, 'color_8': 54, 'color_9': 12}, 2: {'color_0': 15, 'color_1': 40, 'color_2': 64, 'color_3': 60, 'color_4': 45, 'color_5': 1, 'color_6': 29, 'color_7': 95, 'color_8': 67, 'color_9': 54}, 3: {'color_0': 40, 'color_1': 47, 'color_2': 23, 'color_3': 0, 'color_4': 82, 'color_5': 41, 'color_6': 9, 'color_7': 13, 'color_8': 31, 'color_9': 9}, 4: {'color_0': 40, 'color_1': 53, 'color_2': 90, 'color_3': 96, 'color_4': 40, 'color_5': 7, 'color_6': 48, 'color_7': 63, 'color_8': 84, 'color_9': 53}, 5: {'color_0': 40, 'color_1': 53, 'color_2': 90, 'color_3': 63, 'color_4': 27, 'color_5': 63, 'color_6': 68, 'color_7': 97, 'color_8': 26, 'color_9': 55}, 6: {'color_0': 40, 'color_1': 47, 'color_2': 13, 'color_3': 44,

### Next steps:


1. Empericla test your functions at differetn number of buckets to make sure that they are evenly distributed.
2. Wrap the hashing functions in factory classes
3. Make the color iterators do `color_0` for the inital coloring and then `color_1` ... `color_k` for the iterations after than that. (the graph features when will each be a num_buckets long feature K features for K iterations.


# You need a emperical measure of colisions.


In [358]:
import numpy as np
import pandas as pd
import networkx as nx


class ColorRefinement:


  @staticmethod
  def create_color_hash_function(num_buckets: int):

    def get_next_node_color(node_color:int, neighbor_colors:list):
      pre_hash_color = str(node_color) + str(np.prod(neighbor_colors))
      new_color = hash(pre_hash_color) % num_buckets
      return new_color


    hash_function = get_next_node_color
    return hash_function

  @staticmethod
  def convert_node_view_to_node_color_dict(node_data: nx.classes.reportviews.NodeDataView, num_buckets: int)-> dict:
    """
        Set the inital color of each of the nodes based on hashing the node attributes and a number of buckets. 

        Returns a dictionary of inital colors to make the nodes
    """

    node_data_as_dict = dict(node_data)
    node_names = list(node_data_as_dict.keys())
    node_starting_colors = dict()

    for index, node_num in enumerate(node_names):
      node_attributes = tuple(sorted(list(a[node_names[index]].items())))
      inital_node_color = hash(node_attributes) % num_buckets
      node_starting_colors[node_num] = inital_node_color

    return node_starting_colors
  
  @staticmethod
  def create_inital_color_graph(graph: nx.classes.graph.Graph, num_buckets:int):

    starting_colors = convert_node_view_to_node_color_dict(graph.nodes.data(True), num_buckets)
    color_graph = copy.deepcopy(graph)
    nx.set_node_attributes(color_graph, starting_colors, name="color_0")

    # remove unneeded tags
    for index, node in enumerate(color_graph.nodes.data(True)):
      node_attributes = list(color_graph.nodes[index].keys())
      attributes_to_remove = [a for a in node_attributes if a !='color_0']
      [color_graph.nodes[index].pop(a) for a in attributes_to_remove]

    return color_graph

  @staticmethod
  def next_iteration_of_color_graph(color_graph, iter_num:int, color_hashing_function):
    prev_color_key = f'color_{iter_num-1}'
    
    # do an iteration
    updated_colors = dict()
    for index, current_node in enumerate(color_graph.nodes.data(True)):
      neighbor_nodes = [n for n in color_graph.neighbors(index)]
      current_color = color_graph.nodes[current_node[0]]
      neighbor_colors = ([color_graph.nodes[n][prev_color_key] for n in neighbor_nodes])
      updated_colors[index] = color_hashing_function(current_color,neighbor_colors)

    nx.set_node_attributes(color_graph, updated_colors, name=f'color_{iter_num}')

  @staticmethod
  def compute_K_color_refinements(G: nx.classes.graph.Graph, K:int, num_buckets:int):
    """
    Given a graph G and a hash_function f, compute K iterations of the color refinement algorithm (link to algo)
    """
    color_hashing_function = cr.create_color_hash_function(num_buckets)
    color_graph = cr.create_inital_color_graph(G, num_buckets)

    for iter_num in range(1,K):
      cr.next_iteration_of_color_graph(color_graph, iter_num=iter_num, color_hashing_function=color_hashing_function)

    return color_graph

  @staticmethod
  def extact_node_labels():
    pass


cr = ColorRefinement()

fully_colored_graph = cr.compute_K_color_refinements(G=g,K=10,num_buckets=1000)
a = fully_colored_graph.nodes.data(True)
a

NodeDataView({0: {'color_0': 229, 'color_1': 560, 'color_2': 251, 'color_3': 310, 'color_4': 725, 'color_5': 102, 'color_6': 44, 'color_7': 416, 'color_8': 571, 'color_9': 136}, 1: {'color_0': 264, 'color_1': 473, 'color_2': 948, 'color_3': 357, 'color_4': 115, 'color_5': 620, 'color_6': 765, 'color_7': 726, 'color_8': 318, 'color_9': 22}, 2: {'color_0': 515, 'color_1': 430, 'color_2': 121, 'color_3': 420, 'color_4': 696, 'color_5': 344, 'color_6': 219, 'color_7': 186, 'color_8': 797, 'color_9': 194}, 3: {'color_0': 825, 'color_1': 265, 'color_2': 905, 'color_3': 518, 'color_4': 779, 'color_5': 177, 'color_6': 140, 'color_7': 984, 'color_8': 364, 'color_9': 681}, 4: {'color_0': 403, 'color_1': 571, 'color_2': 376, 'color_3': 338, 'color_4': 98, 'color_5': 103, 'color_6': 211, 'color_7': 760, 'color_8': 826, 'color_9': 295}, 5: {'color_0': 889, 'color_1': 263, 'color_2': 238, 'color_3': 734, 'color_4': 643, 'color_5': 794, 'color_6': 628, 'color_7': 274, 'color_8': 948, 'color_9': 425},

In [359]:
node_num = 0

first_node_labels = zip(list(list(a)[node_num][1].keys()), list(list(a)[node_num][1].values()))
node_labels = list(first_node_labels)

df = pd.DataFrame(node_labels, columns=['iteration_number', f'Node_{node_num}'])
df.set_index('iteration_number', inplace=True)
df

,Node_0
iteration_number,
color_0,229
color_1,560
color_2,251
color_3,310
color_4,725
color_5,102
color_6,44
color_7,416
color_8,571


In [ ]:
def extact_node_colors(node)-> tuple:
  labels = list(zip(list(node[1].keys()), list(node[1].values()))) # ugly as heck
  return labels


In [ ]:
labels = extract_node_colors(a[0])
first_df = pd.DataFrame(node_labels, columns=['iteration_number', f'Node_{node_num}'])
first_df.set_index('iteration_number', inplace=True)
first_df

for node_num in range(1,50):
  try:
    labels = extract_node_colors(a[node_num])
    df = pd.DataFrame(node_labels, columns=['iteration_number', f'Node_{node_num}'])
    df.set_index('iteration_number', inplace=True)
    frist_df = pd.concat([df,first_df], axis=1)

  except:
    print(f'broke on {node_num}')
# untested

In [ ]:
def create_node_color_df(color_graph) -> pd.DataFrame:
  """
    untested
  """
  a =  color_graph.nodes.data(True)

  labels = extract_node_colors(a[0])
  first_df = pd.DataFrame(node_labels, columns=['iteration_number', f'Node_{node_num}'])
  first_df.set_index('iteration_number', inplace=True)
  first_df

  for node_num in range(1,len(a)):  
    try:
      labels = extract_node_colors(a[node_num])
      df = pd.DataFrame(node_labels, columns=['iteration_number', f'Node_{node_num}'])
      df.set_index('iteration_number', inplace=True)
      first_df = pd.concat([df,first_df], axis=1)

    except:
      print(f'broke on {node_num}')